In [ ]:
import yfinance as yf 
import pandas as pd 
import os 
from data_loader import * 

# top 20 SP500 stocks  
tickers = [
    'AAPL',   # Apple Inc.
    'MSFT',   # Microsoft Corporation
    'GOOGL',  # Alphabet Inc. (Google)
    'AMZN',   # Amazon.com Inc.
    'NVDA',   # NVIDIA Corporation
    'META',   # Meta Platforms Inc. (Facebook)
    'TSLA',   # Tesla Inc.
    'NFLX',   # Netflix Inc.
    'UNH',    # UnitedHealth Group Inc.
    'JNJ',    # Johnson & Johnson
    'V',      # Visa Inc.
    'JPM',    # JPMorgan Chase & Co.
    'WMT',    # Walmart Inc.
    'MA',     # Mastercard Inc.
    'PG',     # Procter & Gamble Co.
    'HD',     # The Home Depot Inc.
    'DIS',    # The Walt Disney Company
    'BAC',    # Bank of America Corp.
    'XOM',    # Exxon Mobil Corporation
    'CVX'     # Chevron Corporation
]

stock_data = get_data(tickers) 

# Remember to create your own data folder first
for ticker, df in stock_data.items():  
    if not os.path.exists(f'data/{ticker}.csv'):
        df.to_csv(f'data/{ticker}.csv')  



In [ ]:
# Get the data from the CSV files
stock_data = {}
for ticker in tickers:
    df = pd.read_csv(f'data/{ticker}.csv', skiprows=2, index_col='Date', parse_dates=True)
    # Rename the first column to 'Date' if needed and set proper column names
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    stock_data[ticker] = df


In [ ]:
stock_data = process_data_with_indicators(stock_data)
stock_data['AAPL'].loc['2024-01-01':'2026-01-01']

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


In [ ]:
# Plot all tickers 
plt.figure(figsize=(16, 10))
for ticker in stock_data.keys(): 
    plt_data = stock_data[ticker].loc['2024-01-01':'2025-01-01']
    plt.plot(plt_data.index, plt_data['Close'], label=ticker)
plt.legend()    
plt.show()

In [ ]:
def plot_trading_chart(df, ticker, indicators, start_date=None, end_date=None):
    if start_date:
        df = df.loc[start_date:]
    if end_date:
        df = df.loc[:end_date]
    
    # Number of subplots: 1 for price + 1 for each indicator
    n_plots = 1 + len(indicators)
    height_ratios = [3] + [1] * len(indicators)
    
    fig, axes = plt.subplots(n_plots, 1, figsize=(16, 4 + 2*len(indicators)), height_ratios=height_ratios, sharex=True)
    
    # Plot 1: Price with EMAs and Bollinger Bands
    ax_price = axes[0]
    ax_price.plot(df.index, df['Close'], label='Close', color='black', linewidth=1.5)
    ax_price.plot(df.index, df['EMA12'], label='EMA12', color='orange', linewidth=1, linestyle='--')
    ax_price.plot(df.index, df['EMA26'], label='EMA26', color='blue', linewidth=1, linestyle='--')
    
    ax_price.plot(df.index, df['Upper_BB'], label='Upper BB', color='purple', linewidth=1, linestyle='-', alpha=0.7)
    ax_price.plot(df.index, df['Lower_BB'], label='Lower BB', color='purple', linewidth=1, linestyle='-', alpha=0.7)
    ax_price.fill_between(df.index, df['Lower_BB'], df['Upper_BB'], alpha=0.1, color='#9c27b0')
    
    # RSI < 30: Buy signals
    # RSI > 70: Sell signals
    buy_signals = df[df['RSI'] < 30]
    sell_signals = df[df['RSI'] > 70]
    
    # Buy signals: Green triangles
    ax_price.scatter(buy_signals.index, buy_signals['Close'], 
               marker='^', s=100, color='red', label=f'BUY (RSI<30): {len(buy_signals)}',
               edgecolors='black', linewidths=0.5, zorder=5)
    
    # Sell signals: Red triangles
    ax_price.scatter(sell_signals.index, sell_signals['Close'], 
               marker='v', s=100, color='lime', label=f'SELL (RSI>70): {len(sell_signals)}',
               edgecolors='black', linewidths=0.5, zorder=5)
    
    # Plot 2: RSI
    ax_rsi = axes[1]
    ax_rsi.plot(df.index, df['RSI'], color='green', linewidth=1.2)
    ax_rsi.set_ylabel('RSI', fontsize=11)
    # RSI levels (Upper and Lower)
    ax_rsi.axhline(y=70, color='red', linestyle='--', linewidth=0.8, alpha=0.7)
    ax_rsi.axhline(y=30, color='blue', linestyle='--', linewidth=0.8, alpha=0.7)
    ax_rsi.set_ylim(0, 100)
    ax_rsi.grid(alpha=0.3)
    
    # Plot 3: MACD
    ax_macd = axes[2]
    ax_macd.plot(df.index, df['MACD'], color='navy', linewidth=1.2)
    ax_macd.set_ylabel('MACD', fontsize=11)
    ax_macd.axhline(y=0, color='gray', linestyle='-', linewidth=0.8, alpha=0.7)
    ax_macd.grid(alpha=0.3)
    
    ax_price.set_ylabel('Price', fontsize=11)
    ax_price.legend(loc='upper left', fontsize=9)
    ax_price.set_title(f'{ticker} - Trading Chart', fontsize=16, fontweight='bold')
    ax_price.grid(alpha=0.3)

    # Plot 4: Relative Volume
    ax_rv = axes[3]
    ax_rv.plot(df.index, df['RVOL'], color='orange', linewidth=1.2)
    ax_rv.set_ylabel('Relative Volume', fontsize=11)
    ax_rv.set_ylim(0, 3.5)
    ax_rv.grid(alpha=0.3)
    ax_rv.set_xlabel('Date', fontsize=11)
    ax_rv.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    
    plt.tight_layout()
    return axes


plot_trading_chart(stock_data['AAPL'], 'AAPL', indicators=['RSI', 'MACD', 'RVOL'], start_date='2024-01-01', end_date='2024-07-01')